In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import cv2 #to install it -> !pip install opencv-python
from tensorflow import keras
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras import models
from pathlib import Path
from tensorflow.keras.optimizers import SGD

In [2]:
# This function slipts the dataset into train and test folders:
def prepare_data(filepath, src,dest):
    classes_images = defaultdict(list)
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')

    for food in classes_images.keys():
        print("\nCopying images into ",food)
        if not os.path.exists(os.path.join(dest,food)):
            os.makedirs(os.path.join(dest,food))
        for i in classes_images[food]:
            copy(os.path.join(src,food,i), os.path.join(dest,food,i))
    print("Copying Done!")

In [5]:
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
print("Creating train data...")
prepare_data('../food-images/image_plates/train.txt', '../food-images/image_plates', '../food-images/image_plates/train')

Creating train data...

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  breakfast_burrito

Copying images into  caesar_salad

Copying images into  caprese_salad

Copying images into  ceviche

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  club_sandwich

Copying images into  dumplings

Copying images into  edamame

Copying images into  falafel

Copying images into  french_fries

Copying images into  fried_calamari

Copying images into  fried_rice

Copying images into  greek_salad

Copying images into  grilled_cheese_sandwich

Copying images into  guacamole

Copying images into  gyoza

Copying images into  hamburger

Copying images into  hot_dog

Copying images into  hummus

Copying images into  lasagna

Copying images into  miso_soup

Copying images into  mussels

Copying images into  nachos

Copying images into  omelette

Copying images into  onion_rings

Copy

In [6]:
# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
print("Creating test data...")
prepare_data('../food-images/image_plates/test.txt', '../food-images/image_plates', '../food-images/image_plates/test')

Creating test data...

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  breakfast_burrito

Copying images into  caesar_salad

Copying images into  caprese_salad

Copying images into  ceviche

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  club_sandwich

Copying images into  dumplings

Copying images into  edamame

Copying images into  falafel

Copying images into  french_fries

Copying images into  fried_calamari

Copying images into  fried_rice

Copying images into  greek_salad

Copying images into  grilled_cheese_sandwich

Copying images into  guacamole

Copying images into  gyoza

Copying images into  hamburger

Copying images into  hot_dog

Copying images into  hummus

Copying images into  lasagna

Copying images into  miso_soup

Copying images into  mussels

Copying images into  nachos

Copying images into  omelette

Copying images into  onion_rings

Copyi

In [7]:
# Check how many files are in the train folder
print("Total number of samples in train folder")
train_total_samples = !find train -type d -or -type f -printf '.' | wc -c
train_total_samples = int(train_total_samples[0])
train_total_samples

Total number of samples in train folder


ValueError: invalid literal for int() with base 10: 'find: ‘train’: No such file or directory'

In [8]:
# Check how many files are in the test folder
print("Total number of samples in test folder")
test_total_samples = !find test -type d -or -type f -printf '.' | wc -c
test_total_samples = int(test_total_samples[0])
test_total_samples

Total number of samples in test folder


ValueError: invalid literal for int() with base 10: 'find: ‘test’: No such file or directory'

# Model classification training

In [ ]:
#To know where the last dimension goes:
K.image_data_format()

In [ ]:
#To know how many folders of images do we have, this will be our number of clases:
a = !ls food-101/food-101/images
len(a)

# PLATES - Model classification training

In [11]:
#Clearing the session removes all the nodes left over from previous models, freeing memory and preventing slowdown.
K.clear_session()


#Defining number of clases, num of pixels, batch_size-> dataset divided in blocks to train
#Epochs->num of times the model runs the same dataset
NUM_CLASSES = 50
IMG_ROWS, IMG_COLS = 128, 128
BATCH_SIZE = 16
EPOCHS =20



#Folder to take train and test data into:
train_data_folder = '../food-images/image_plates/train'
validation_data_folder = '../food-images/image_plates/test'


#defining the total number of train and validation images:
num_train_samples = 37500
num_validation_samples = 12500

train_datagenerator = ImageDataGenerator(rescale=1. / 255,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True)


test_datagenerator = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagenerator.flow_from_directory(train_data_folder,
                                                         target_size=(IMG_ROWS, IMG_COLS),
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical')


test_generator = test_datagenerator.flow_from_directory(validation_data_folder,
                                                       target_size=(IMG_ROWS, IMG_COLS),
                                                       batch_size=BATCH_SIZE,
                                                       class_mode='categorical')


# create the base pre-trained model:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) #use to reduce overfiting


predictions = Dense(NUM_CLASSES, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_45class_salads.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('traning_total45_salads.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = num_train_samples // BATCH_SIZE,
                    validation_data=test_generator,
                    validation_steps=num_validation_samples // BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_45class_salads.hdf5')

Found 37500 images belonging to 50 classes.
Found 12500 images belonging to 50 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2343 steps, validate for 781 steps
Epoch 1/20
2342/2343 [============================>.] - ETA: 0s - loss: 4.2140 - accuracy: 0.0792
Epoch 00001: val_loss improved from inf to 3.69098, saving model to best_model_45class_salads.hdf5
2343/2343 [==============================] - 1986s 848ms/step - loss: 4.2139 - accuracy: 0.0792 - val_loss: 3.6910 - val_accuracy: 0.2214
Epoch 2/20
2342/2343 [============================>.] - ETA: 0s - loss: 3.4397 - accuracy: 0.2526
Epoch 00002: val_loss improved from 3.69098 to 2.82870, saving model to best_model_45class_salads.hdf5
2343/2343 [==============================] - 1973s 842ms/step - loss: 3.4397 - accuracy: 0.2526 - val_loss: 2.8287 - val_accuracy: 0.3850
Epoch 3/20
2342/2343 [============================>.] - ETA: 0s - l

# SALADS - Model classification training

In [ ]:
#Clearing the session removes all the nodes left over from previous models, freeing memory and preventing slowdown.
K.clear_session()


#Defining number of clases, num of pixels, batch_size-> dataset divided in blocks to train
#Epochs->num of times the model runs the same dataset
NUM_CLASSES = 5
IMG_ROWS, IMG_COLS = 128, 128
BATCH_SIZE = 16
EPOCHS =20

labels = ['beet_salad', 'caesar_salad', 'caprese_salad', 'greek_salad', 'seaweed_salad']

#Folder to take train and test data into:
train_data_folder = 'food-images/image_salads/train'
validation_data_folder = 'food-images/image_salads/test'


#defining the total number of train and validation images:
num_train_samples = 3750
num_validation_samples = 1250

train_datagenerator = ImageDataGenerator(rescale=1. / 255,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True)


test_datagenerator = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagenerator.flow_from_directory(train_data_folder,
                                                         target_size=(IMG_ROWS, IMG_COLS),
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical')


test_generator = test_datagenerator.flow_from_directory(validation_data_folder,
                                                       target_size=(IMG_ROWS, IMG_COLS),
                                                       batch_size=BATCH_SIZE,
                                                       class_mode='categorical')


# create the base pre-trained model:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) #use to reduce overfiting


predictions = Dense(NUM_CLASSES, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_salads.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('traning_salads.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = num_train_samples // BATCH_SIZE,
                    validation_data=test_generator,
                    validation_steps=num_validation_samples // BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_salads.hdf5')

# DESERTS - Model classification training

In [ ]:
#Clearing the session removes all the nodes left over from previous models, freeing memory and preventing slowdown.
K.clear_session()


#Defining number of clases, num of pixels, batch_size-> dataset divided in blocks to train
#Epochs->num of times the model runs the same dataset
NUM_CLASSES = 9
IMG_ROWS, IMG_COLS = 128, 128
BATCH_SIZE = 16
EPOCHS = 18



#Folder to take train and test data into:
train_data_folder = 'food-images/image_desert/train'
validation_data_folder = 'food-images/image_desert/test'


#defining the total number of train and validation images:
num_train_samples = 6750
num_validation_samples = 2250

train_datagenerator = ImageDataGenerator(rescale=1. / 255,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True)


test_datagenerator = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagenerator.flow_from_directory(train_data_folder,
                                                         target_size=(IMG_ROWS, IMG_COLS),
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical')


test_generator = test_datagenerator.flow_from_directory(validation_data_folder,
                                                       target_size=(IMG_ROWS, IMG_COLS),
                                                       batch_size=BATCH_SIZE,
                                                       class_mode='categorical')


# create the base pre-trained model:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) #use to reduce overfiting


predictions = Dense(NUM_CLASSES, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_9desert.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('model9desert.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = num_train_samples // BATCH_SIZE,
                    validation_data=test_generator,
                    validation_steps=num_validation_samples // BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_9desert.hdf5')